In [1]:
from fastai.vision import *
import numpy as np
import torch.utils.data as data_utils
import torch
from sklearn import preprocessing
import cv2
from torchvision.utils import make_grid

## Custom Dataset

In [2]:
class NumbersDataset():
    def __init__(self, inputs, labels):
        self.X = inputs
        self.y = labels
        self.c = 15


    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_train = self.X
        img_mask = self.y
        return img_train, img_mask

In [3]:
class CustomTensorDataset(Dataset):

    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform
        self.c = 15

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        c = 15

        return x, y, c

    def __len__(self):
        return self.tensors[0].size(0)

## Train data

In [2]:
X_train = np.load("../data/generator/generated_data.npy")
Y_train = np.load("../data/generator/generated_data_labels.npy")

In [3]:
X_train = np.repeat(X_train[..., np.newaxis], 3, -1)
X_train.shape

(141750, 100, 100, 3)

In [4]:
X_train = np.transpose(X_train, (0, 3, 1, 2))
X_train.shape

(141750, 3, 100, 100)

In [5]:
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
#le.inverse_transform(Y_train)

In [6]:
Y_train.shape

(141750,)

In [8]:
X_train = torch.from_numpy(X_train)
Y_train = torch.Tensor(Y_train)

In [9]:
X_train.size(), Y_train.size()

(torch.Size([141750, 100, 100, 3]), torch.Size([141750]))

In [10]:
#train_ds = data_utils.TensorDataset(X_train, Y_train)
#train_ds = CustomTensorDataset(tensors = (X_train, Y_train))
train_ds = NumbersDataset(X_train, Y_train)

## Test data

In [11]:
test_set = []
test_set_labels = []

#Read in sample.
for filename in os.listdir("./test_data/"):
    img = cv2.imread("./test_data/" + filename,0)
    name = re.findall('([a-zA-Z_ ]*)\d*.*', filename)[0]
    #Preprocess image by removing blank columns and rows from edges
    img = img[np.argwhere(np.amin(img,axis=1) < 120)[0][0]:np.argwhere(np.amin(img,axis=1) < 120)[-1][0],:]
    img = img[:,np.argwhere(np.amin(img,axis=0) < 120)[0][0]:np.argwhere(np.amin(img,axis=0) < 120)[-1][0]]
    img = cv2.resize(img, [100,100])
    test_set.append(img)
    test_set_labels.append(name)

In [12]:
X_test = np.array(test_set)

In [13]:
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
X_test.shape

(585, 100, 100, 3)

In [14]:
Y_test = le.transform(np.array(test_set_labels))

In [15]:
X_test = torch.from_numpy(np.array(X_test))
Y_test = torch.Tensor(Y_test)

In [16]:
X_test.size(), Y_test.size()

(torch.Size([585, 100, 100, 3]), torch.Size([585]))

In [17]:
#test_ds = data_utils.TensorDataset(X_test, Y_test)
#test_ds = CustomTensorDataset(tensors = (X_test, Y_test))
test_ds = NumbersDataset(X_test, Y_test)

## Dataloader

In [18]:
dataloader_train = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0)
dataloader_test = DataLoader(test_ds, batch_size=4, shuffle=True, num_workers=0)

In [19]:
data = ImageDataBunch.create(dataloader_train, dataloader_test) # , classes=["advance_to_contract", "attack", "block"]
#data = ImageDataBunch(X_train, Y_train)
#data = ImageDataBunch(train_ds, test_ds)

In [20]:
data

ImageDataBunch;

Train: <__main__.NumbersDataset object at 0x000001FF502EDDF0>;

Valid: <__main__.NumbersDataset object at 0x000001FF502ED3A0>;

Test: None

## Array ( another experiment )


In [7]:
class ArrayImageList(ImageList):
    @classmethod
    def from_numpy(cls, numpy_array):
        return cls(items=numpy_array)

    def label_from_array(self, array, label_cls=None, **kwargs):
        return self._label_from_list(array,label_cls=label_cls,**kwargs)

    def get(self, i):
        n = self.items[i]
        n = torch.tensor(n).float()
        return Image(n)

class ArrayImageDataBunch(ImageDataBunch):
    @classmethod
    def from_numpy(cls, numpy_array, labels, train_size=0.8, valid_size=0.2, **kwargs:Any)->'ImageDataBunch':
        "Create from numpy array"
        src = (ArrayImageList.from_numpy(numpy_array)
        .split_subsets(train_size=train_size, valid_size=valid_size)
        .label_from_array(labels))
        return cls.create_from_ll(src, **kwargs)

In [8]:
data = (ArrayImageDataBunch.from_numpy(X_train[0:500], Y_train[0:500]))

In [9]:
data

ImageDataBunch;

Train: LabelList (400 items)
x: ArrayImageList
Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100)
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (100 items)
x: ArrayImageList
Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100),Image (3, 100, 100)
y: CategoryList
0,0,0,0,0
Path: .;

Test: None

## Learning

In [11]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.confusion_matrix()